# IMDb Review Analysis: Basic Statistics

This notebook provides foundational statistical analysis of the review dataset before diving into NLP.

## Sections:
1. Dataset Overview
2. Rating Analysis
3. Temporal Analysis
4. Review Characteristics
5. Helpful Votes Analysis
6. Spoiler Analysis
7. Cross-Movie Comparison

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path

# Set style for better-looking plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load Data

In [ ]:
# Load the CSV
csv_path = Path('../Data/all_reviews.csv')
df = pd.read_csv(csv_path)

# Convert date column to datetime
df['Review_Date'] = pd.to_datetime(df['Review_Date'])

# Convert Spoiler_Flag to boolean
df['Spoiler_Flag'] = (df['Spoiler_Flag'].astype(str) == 'True')

# Handle NULL values
df = df.replace('NULL', np.nan)

# Display basic info
print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]} reviews × {df.shape[1]} columns")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())
print(f"\nPercentage of missing values:")
print((df.isnull().sum() / len(df) * 100).round(2))

## 2. Dataset Overview

In [ ]:
# Overall statistics
print("="*60)
print("DATASET OVERVIEW")
print("="*60)

total_reviews = len(df)
total_movies = df['Movie_Title'].nunique()
total_reviewers = df['Reviewer'].nunique()

print(f"\nTotal Reviews: {total_reviews:,}")
print(f"Total Movies: {total_movies}")
print(f"Total Unique Reviewers: {total_reviewers:,}")

# Date range
if df['Review_Date'].notna().any():
    earliest_date = df['Review_Date'].min()
    latest_date = df['Review_Date'].max()
    print(f"\nDate Range: {earliest_date.strftime('%Y-%m-%d')} to {latest_date.strftime('%Y-%m-%d')}")
    print(f"Time Span: {(latest_date - earliest_date).days} days ({(latest_date - earliest_date).days / 365.25:.1f} years)")

# Reviews per movie
reviews_per_movie = df.groupby('Movie_Title').size()
print(f"\nReviews per Movie:")
print(f"  Mean: {reviews_per_movie.mean():.1f}")
print(f"  Median: {reviews_per_movie.median():.0f}")
print(f"  Min: {reviews_per_movie.min()}")
print(f"  Max: {reviews_per_movie.max()}")

print(f"\nMovies in dataset:")
for movie, count in reviews_per_movie.items():
    print(f"  {movie}: {count} reviews")

## 3. Rating Analysis

In [ ]:
# Overall rating statistics
print("="*60)
print("RATING ANALYSIS")
print("="*60)

print(f"\nOverall Rating Statistics:")
print(f"  Mean: {df['Rating'].mean():.2f}/10")
print(f"  Median: {df['Rating'].median():.1f}/10")
print(f"  Mode: {df['Rating'].mode().values[0]}/10")
print(f"  Std Dev: {df['Rating'].std():.2f}")
print(f"  Min: {df['Rating'].min()}/10")
print(f"  Max: {df['Rating'].max()}/10")

# Rating distribution
print(f"\nRating Distribution:")
rating_counts = df['Rating'].value_counts().sort_index()
for rating, count in rating_counts.items():
    percentage = (count / len(df)) * 100
    bar = '█' * int(percentage / 2)
    print(f"  {rating}/10: {count:3d} reviews ({percentage:5.1f}%) {bar}")

In [ ]:
# Visualize rating distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['Rating'], bins=10, range=(0.5, 10.5), edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Rating (1-10)')
axes[0].set_ylabel('Number of Reviews')
axes[0].set_title('Rating Distribution (All Movies)')
axes[0].axvline(df['Rating'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["Rating"].mean():.2f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot by movie
df.boxplot(column='Rating', by='Movie_Title', ax=axes[1])
axes[1].set_xlabel('Movie')
axes[1].set_ylabel('Rating (1-10)')
axes[1].set_title('Rating Distribution by Movie')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')  # Rotate labels
plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

In [ ]:
# Polarization analysis (high std dev = polarizing)
print("\nPolarization Index (by standard deviation):")
polarization = df.groupby('Movie_Title')['Rating'].std().sort_values(ascending=False)
for movie, std in polarization.items():
    print(f"  {movie}: {std:.2f} (higher = more polarizing)")

## 4. Review Characteristics

In [ ]:
# Review length analysis
print("="*60)
print("REVIEW LENGTH ANALYSIS")
print("="*60)

print(f"\nWord Count Statistics:")
print(f"  Mean: {df['Review_Length'].mean():.1f} words")
print(f"  Median: {df['Review_Length'].median():.0f} words")
print(f"  Std Dev: {df['Review_Length'].std():.1f} words")
print(f"  Min: {df['Review_Length'].min()} words")
print(f"  Max: {df['Review_Length'].max()} words")

# Quartiles
quartiles = df['Review_Length'].quantile([0.25, 0.5, 0.75])
print(f"\nQuartiles:")
print(f"  25th percentile: {quartiles[0.25]:.0f} words")
print(f"  50th percentile: {quartiles[0.50]:.0f} words")
print(f"  75th percentile: {quartiles[0.75]:.0f} words")

# Title analysis
has_title = df['Review_Title'].notna().sum()
no_title = df['Review_Title'].isna().sum()
print(f"\nReview Titles:")
print(f"  With title: {has_title} ({has_title/len(df)*100:.1f}%)")
print(f"  Without title: {no_title} ({no_title/len(df)*100:.1f}%)")

In [ ]:
# Visualize review length distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histogram of review lengths
axes[0, 0].hist(df['Review_Length'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Word Count')
axes[0, 0].set_ylabel('Number of Reviews')
axes[0, 0].set_title('Review Length Distribution')
axes[0, 0].axvline(df['Review_Length'].mean(), color='red', linestyle='--', label=f'Mean: {df["Review_Length"].mean():.0f}')
axes[0, 0].legend()

# Box plot: Length by rating bucket
df['Rating_Bucket'] = pd.cut(df['Rating'], bins=[0, 3, 6, 8, 10], labels=['1-3 (Bad)', '4-6 (Mixed)', '7-8 (Good)', '9-10 (Great)'])
df.boxplot(column='Review_Length', by='Rating_Bucket', ax=axes[0, 1])
axes[0, 1].set_xlabel('Rating Bucket')
axes[0, 1].set_ylabel('Word Count')
axes[0, 1].set_title('Review Length by Rating Bucket')
plt.suptitle('')

# Scatter: Rating vs Length
axes[1, 0].scatter(df['Rating'], df['Review_Length'], alpha=0.3)
axes[1, 0].set_xlabel('Rating (1-10)')
axes[1, 0].set_ylabel('Word Count')
axes[1, 0].set_title('Rating vs. Review Length')
axes[1, 0].grid(True, alpha=0.3)

# Average length by rating
avg_length_by_rating = df.groupby('Rating')['Review_Length'].mean()
axes[1, 1].bar(avg_length_by_rating.index, avg_length_by_rating.values, edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Rating (1-10)')
axes[1, 1].set_ylabel('Average Word Count')
axes[1, 1].set_title('Average Review Length by Rating')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Correlation
correlation = df['Rating'].corr(df['Review_Length'])
print(f"\nCorrelation between Rating and Review Length: {correlation:.3f}")
if abs(correlation) < 0.1:
    print("  → Very weak correlation")
elif abs(correlation) < 0.3:
    print("  → Weak correlation")
else:
    print("  → Moderate correlation")

## 5. Temporal Analysis

In [ ]:
# Temporal analysis
print("="*60)
print("TEMPORAL ANALYSIS")
print("="*60)

# Extract year and month
df['Year'] = df['Review_Date'].dt.year
df['Month'] = df['Review_Date'].dt.month
df['Year_Month'] = df['Review_Date'].dt.to_period('M')

# Reviews per year
print("\nReviews per Year:")
reviews_per_year = df.groupby('Year').size().sort_index()
for year, count in reviews_per_year.items():
    print(f"  {year}: {count} reviews")

# Average rating over time
print("\nAverage Rating by Year:")
avg_rating_per_year = df.groupby('Year')['Rating'].mean().round(2)
for year, avg_rating in avg_rating_per_year.items():
    print(f"  {year}: {avg_rating}/10")

In [ ]:
# Visualize temporal trends
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Reviews over time
reviews_over_time = df.groupby('Year_Month').size()
axes[0].plot(reviews_over_time.index.astype(str), reviews_over_time.values, marker='o', linewidth=2)
axes[0].set_xlabel('Year-Month')
axes[0].set_ylabel('Number of Reviews')
axes[0].set_title('Review Volume Over Time')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Average rating over time
avg_rating_over_time = df.groupby('Year_Month')['Rating'].mean()
axes[1].plot(avg_rating_over_time.index.astype(str), avg_rating_over_time.values, marker='o', linewidth=2, color='orange')
axes[1].set_xlabel('Year-Month')
axes[1].set_ylabel('Average Rating (1-10)')
axes[1].set_title('Average Rating Over Time')
axes[1].axhline(df['Rating'].mean(), color='red', linestyle='--', label=f'Overall Mean: {df["Rating"].mean():.2f}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Helpful Votes Analysis

In [ ]:
# Helpful votes analysis
print("="*60)
print("HELPFUL VOTES ANALYSIS")
print("="*60)

# Convert to numeric (in case they're strings)
df['Helpful_Votes_Up'] = pd.to_numeric(df['Helpful_Votes_Up'], errors='coerce')
df['Helpful_Votes_Down'] = pd.to_numeric(df['Helpful_Votes_Down'], errors='coerce')

# Calculate total votes and ratio
df['Total_Votes'] = df['Helpful_Votes_Up'] + df['Helpful_Votes_Down']
df['Upvote_Ratio'] = df['Helpful_Votes_Up'] / (df['Total_Votes'] + 1)  # +1 to avoid division by zero

print(f"\nUpvote Statistics:")
print(f"  Mean: {df['Helpful_Votes_Up'].mean():.1f}")
print(f"  Median: {df['Helpful_Votes_Up'].median():.0f}")
print(f"  Max: {df['Helpful_Votes_Up'].max():.0f}")

print(f"\nDownvote Statistics:")
print(f"  Mean: {df['Helpful_Votes_Down'].mean():.1f}")
print(f"  Median: {df['Helpful_Votes_Down'].median():.0f}")
print(f"  Max: {df['Helpful_Votes_Down'].max():.0f}")

print(f"\nTotal Votes Statistics:")
print(f"  Mean: {df['Total_Votes'].mean():.1f}")
print(f"  Median: {df['Total_Votes'].median():.0f}")

print(f"\nAverage Upvote Ratio: {df['Upvote_Ratio'].mean():.2%}")

# Most helpful reviews
print("\nTop 5 Most Helpful Reviews (by upvotes):")
top_helpful = df.nlargest(5, 'Helpful_Votes_Up')[['Movie_Title', 'Reviewer', 'Rating', 'Helpful_Votes_Up', 'Helpful_Votes_Down', 'Review_Title']]
print(top_helpful.to_string(index=False))

In [ ]:
# Correlation between rating and helpfulness
print("\nCorrelations with Helpful Votes:")
print(f"  Rating vs. Upvotes: {df['Rating'].corr(df['Helpful_Votes_Up']):.3f}")
print(f"  Review Length vs. Upvotes: {df['Review_Length'].corr(df['Helpful_Votes_Up']):.3f}")

# Average upvotes by rating
print("\nAverage Upvotes by Rating:")
avg_upvotes_by_rating = df.groupby('Rating')['Helpful_Votes_Up'].mean().round(1)
for rating, avg_upvotes in avg_upvotes_by_rating.items():
    print(f"  {rating}/10: {avg_upvotes} upvotes")

In [ ]:
# Visualize helpful votes
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Upvotes distribution
axes[0, 0].hist(df['Helpful_Votes_Up'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Upvotes')
axes[0, 0].set_ylabel('Number of Reviews')
axes[0, 0].set_title('Upvotes Distribution')
axes[0, 0].set_xlim(0, df['Helpful_Votes_Up'].quantile(0.95))  # Trim outliers for better view

# Rating vs Upvotes
avg_upvotes = df.groupby('Rating')['Helpful_Votes_Up'].mean()
axes[0, 1].bar(avg_upvotes.index, avg_upvotes.values, edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Rating (1-10)')
axes[0, 1].set_ylabel('Average Upvotes')
axes[0, 1].set_title('Average Upvotes by Rating')
axes[0, 1].grid(True, alpha=0.3)

# Review Length vs Upvotes (scatter)
axes[1, 0].scatter(df['Review_Length'], df['Helpful_Votes_Up'], alpha=0.3)
axes[1, 0].set_xlabel('Review Length (words)')
axes[1, 0].set_ylabel('Upvotes')
axes[1, 0].set_title('Review Length vs. Upvotes')
axes[1, 0].grid(True, alpha=0.3)

# Upvote ratio distribution
axes[1, 1].hist(df['Upvote_Ratio'], bins=20, edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Upvote Ratio (Upvotes / Total Votes)')
axes[1, 1].set_ylabel('Number of Reviews')
axes[1, 1].set_title('Upvote Ratio Distribution')
axes[1, 1].axvline(df['Upvote_Ratio'].mean(), color='red', linestyle='--', label=f'Mean: {df["Upvote_Ratio"].mean():.2%}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 7. Spoiler Analysis

In [ ]:
# Spoiler flag analysis
print("="*60)
print("SPOILER ANALYSIS")
print("="*60)

spoiler_count = df['Spoiler_Flag'].sum()
non_spoiler_count = (~df['Spoiler_Flag']).sum()

print(f"\nSpoiler Distribution:")
print(f"  With spoilers: {spoiler_count} ({spoiler_count/len(df)*100:.1f}%)")
print(f"  Without spoilers: {non_spoiler_count} ({non_spoiler_count/len(df)*100:.1f}%)")

# Compare spoiler vs non-spoiler reviews
spoiler_stats = df.groupby('Spoiler_Flag').agg({
    'Rating': ['mean', 'std'],
    'Review_Length': ['mean', 'median'],
    'Helpful_Votes_Up': 'mean'
}).round(2)

print("\nSpoiler vs. Non-Spoiler Comparison:")
print(spoiler_stats)

In [ ]:
# Visualize spoiler comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Rating comparison
df.boxplot(column='Rating', by='Spoiler_Flag', ax=axes[0])
axes[0].set_xlabel('Spoiler Flag')
axes[0].set_ylabel('Rating (1-10)')
axes[0].set_title('Rating: Spoiler vs. Non-Spoiler')
axes[0].set_xticklabels(['No Spoiler', 'Spoiler'])
plt.suptitle('')

# Length comparison
df.boxplot(column='Review_Length', by='Spoiler_Flag', ax=axes[1])
axes[1].set_xlabel('Spoiler Flag')
axes[1].set_ylabel('Word Count')
axes[1].set_title('Review Length: Spoiler vs. Non-Spoiler')
axes[1].set_xticklabels(['No Spoiler', 'Spoiler'])

# Helpfulness comparison
df.boxplot(column='Helpful_Votes_Up', by='Spoiler_Flag', ax=axes[2])
axes[2].set_xlabel('Spoiler Flag')
axes[2].set_ylabel('Upvotes')
axes[2].set_title('Helpfulness: Spoiler vs. Non-Spoiler')
axes[2].set_xticklabels(['No Spoiler', 'Spoiler'])

plt.tight_layout()
plt.show()

## 8. Cross-Movie Comparison

In [ ]:
# Compare all movies
print("="*60)
print("CROSS-MOVIE COMPARISON")
print("="*60)

movie_comparison = df.groupby('Movie_Title').agg({
    'Review_ID': 'count',
    'Rating': ['mean', 'median', 'std'],
    'Review_Length': 'mean',
    'Helpful_Votes_Up': 'sum',
    'Spoiler_Flag': 'sum'
}).round(2)

# Rename columns for clarity
movie_comparison.columns = ['Total_Reviews', 'Avg_Rating', 'Median_Rating', 'Rating_StdDev', 
                             'Avg_Length', 'Total_Upvotes', 'Spoiler_Count']

# Sort by average rating
movie_comparison = movie_comparison.sort_values('Avg_Rating', ascending=False)

print("\nMovie Comparison Summary:")
print(movie_comparison)

# Identify most/least polarizing
print("\nMost Polarizing Movie (highest std dev):")
most_polarizing = movie_comparison['Rating_StdDev'].idxmax()
print(f"  {most_polarizing} (std dev: {movie_comparison.loc[most_polarizing, 'Rating_StdDev']})")

print("\nLeast Polarizing Movie (lowest std dev):")
least_polarizing = movie_comparison['Rating_StdDev'].idxmin()
print(f"  {least_polarizing} (std dev: {movie_comparison.loc[least_polarizing, 'Rating_StdDev']})")

In [ ]:
# Visualize movie comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Average rating by movie
movie_comparison['Avg_Rating'].plot(kind='barh', ax=axes[0, 0], color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('Average Rating (1-10)')
axes[0, 0].set_ylabel('Movie')
axes[0, 0].set_title('Average Rating by Movie')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Total reviews by movie
movie_comparison['Total_Reviews'].plot(kind='barh', ax=axes[0, 1], color='lightcoral', edgecolor='black')
axes[0, 1].set_xlabel('Number of Reviews')
axes[0, 1].set_ylabel('Movie')
axes[0, 1].set_title('Total Reviews by Movie')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Polarization (std dev) by movie
movie_comparison['Rating_StdDev'].plot(kind='barh', ax=axes[1, 0], color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('Standard Deviation')
axes[1, 0].set_ylabel('Movie')
axes[1, 0].set_title('Polarization by Movie (higher = more divided)')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Average review length by movie
movie_comparison['Avg_Length'].plot(kind='barh', ax=axes[1, 1], color='plum', edgecolor='black')
axes[1, 1].set_xlabel('Average Word Count')
axes[1, 1].set_ylabel('Movie')
axes[1, 1].set_title('Average Review Length by Movie')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 9. Summary & Key Insights

In [ ]:
# Generate summary report
print("="*60)
print("KEY INSIGHTS SUMMARY")
print("="*60)

print(f"\n📊 DATASET:")
print(f"  • {total_reviews:,} reviews across {total_movies} movies")
print(f"  • Date range: {earliest_date.strftime('%Y')} to {latest_date.strftime('%Y')}")

print(f"\n⭐ RATINGS:")
print(f"  • Average rating: {df['Rating'].mean():.2f}/10")
print(f"  • Most common rating: {df['Rating'].mode().values[0]}/10")
print(f"  • Most polarizing movie: {most_polarizing}")

print(f"\n📝 REVIEW CHARACTERISTICS:")
print(f"  • Average length: {df['Review_Length'].mean():.0f} words")
print(f"  • {has_title/len(df)*100:.1f}% have titles")
print(f"  • {spoiler_count/len(df)*100:.1f}% marked as spoilers")

print(f"\n👍 ENGAGEMENT:")
print(f"  • Average upvotes: {df['Helpful_Votes_Up'].mean():.1f}")
print(f"  • Upvote ratio: {df['Upvote_Ratio'].mean():.1%}")

# Identify patterns
print(f"\n🔍 PATTERNS OBSERVED:")
corr_rating_length = df['Rating'].corr(df['Review_Length'])
if abs(corr_rating_length) > 0.1:
    direction = "longer" if corr_rating_length > 0 else "shorter"
    print(f"  • Higher-rated reviews tend to be {direction}")
else:
    print(f"  • Rating and review length show little correlation")

if df.groupby('Spoiler_Flag')['Review_Length'].mean()[True] > df.groupby('Spoiler_Flag')['Review_Length'].mean()[False]:
    print(f"  • Spoiler reviews are longer on average")
else:
    print(f"  • Non-spoiler reviews are longer on average")

print(f"\n✅ Analysis complete! Ready for NLP phase.")

## Next Steps

Now that we have baseline statistics, we can proceed to:
1. **Sentiment Analysis** - Analyze emotional tone beyond just ratings
2. **Audience Segmentation** - Cluster reviews into distinct audience groups
3. **Topic Modeling** - Extract key themes and talking points
4. **Keyword Extraction** - Identify what people love/hate
5. **Comparative Analysis** - Find patterns across movies